## Road Extraction

In [3]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(r"C:\Users\nasir\OneDrive\Pictures\Screenshots\Screenshot 2025-06-18 090341.png")

if image is None:
    print("Error: Could not load image. Check file path.")
else:
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur with optimized kernel size (reduced from 15x15)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Compute gradients using Scharr operator (more accurate than Sobel for 3x3)
    grad_x = cv2.Scharr(blurred, cv2.CV_16S, 1, 0)
    grad_y = cv2.Scharr(blurred, cv2.CV_16S, 0, 1)
    
    # Compute absolute gradients and combine
    abs_x = cv2.convertScaleAbs(grad_x)
    abs_y = cv2.convertScaleAbs(grad_y)
    gradient = cv2.addWeighted(abs_x, 0.5, abs_y, 0.5, 0)
    
    # Apply adaptive thresholding
    thresholded = cv2.threshold(gradient, 100, 255, cv2.THRESH_BINARY)[1]
    
    # Display result
    cv2.imshow('Edges', thresholded)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Active Contour Model (Snake Algorithm) 
By using Python using OpenCV and NumPy. It is designed to detect object boundaries in a grayscale image by evolving a closed contour based on both image gradients and smoothness constraints.

- alpha: elasticity (low = flexible contour).
- beta: curvature (high = smoother contour).
- gamma: learning rate.
- gradient_threshold: controls sensitivity to edges.
- n_points: number of contour points.


In [4]:
import cv2
import numpy as np
import math

class ActiveContour:
    def __init__(self, image, gradient_threshold=30):
        self.image = cv2.GaussianBlur(image, (9,9), 2)
        self.grad_x = cv2.Sobel(self.image, cv2.CV_64F, 1, 0, ksize=3)
        self.grad_y = cv2.Sobel(self.image, cv2.CV_64F, 0, 1, ksize=3)
        self.grad_mag = np.sqrt(self.grad_x**2 + self.grad_y**2)
        self.gradient_threshold = gradient_threshold
        # Apply threshold to gradient magnitude
        self.grad_mag[self.grad_mag < gradient_threshold] = 0
        self.height, self.width = image.shape
        
    def initialize_contour(self, n_points=30, padding=20):
        """Initialize contour around image edges with flexible point spacing"""
        points = []
        for i in range(n_points):
            angle = 2 * np.pi * i / n_points
            x = int(self.width//2 + (self.width//2 - padding) * np.cos(angle))
            y = int(self.height//2 + (self.height//2 - padding) * np.sin(angle))
            points.append([x,y])
        return np.array(points, dtype=np.float32)
    
    def external_energy(self, x, y):
        """Attraction to edges with directional gradient (thresholded)"""
        x, y = int(round(x)), int(round(y))
        if 0 <= x < self.width and 0 <= y < self.height:
            # Only consider gradients above threshold
            if self.grad_mag[y,x] >= self.gradient_threshold:
                return -self.grad_mag[y,x]  # Negative for minimization
        return 0
    
    def internal_energy(self, points, alpha, beta):
        """Flexible spacing with smoothness constraint"""
        n = len(points)
        if n < 3:
            return 0
        
        # Elasticity term (allows stretching but prevents collapse)
        elastic = 0
        for i in range(n):
            dx = points[(i+1)%n][0] - points[i][0]
            dy = points[(i+1)%n][1] - points[i][1]
            elastic += alpha * (dx**2 + dy**2)
        
        # Curvature term (smoothness)
        curvature = 0
        for i in range(n):
            x_prev, y_prev = points[(i-1)%n]
            x, y = points[i]
            x_next, y_next = points[(i+1)%n]
            curvature += beta * ((x_prev + x_next - 2*x)**2 + (y_prev + y_next - 2*y)**2)
        
        return elastic + curvature
    
    def optimize(self, n_points=30, alpha=0.5, beta=1.0, gamma=0.1, max_iter=200):
        """Gradient descent optimization with adaptive learning"""
        points = self.initialize_contour(n_points)
        energy_history = []
        
        for iteration in range(max_iter):
            # Compute energies
            external = sum(self.external_energy(x,y) for x,y in points)
            internal = self.internal_energy(points, alpha, beta)
            total = external + internal
            energy_history.append(total)
            
            # Gradient descent update
            new_points = np.zeros_like(points)
            for i in range(len(points)):
                # Gradient of external energy (move toward strong edges only)
                x, y = int(round(points[i][0])), int(round(points[i][1]))
                if 0 <= x < self.width and 0 <= y < self.height and self.grad_mag[y,x] >= self.gradient_threshold:
                    gx = self.grad_x[y,x]
                    gy = self.grad_y[y,x]
                else:
                    gx, gy = 0, 0
                
                # Gradient of internal energy
                x_prev, y_prev = points[(i-1)%len(points)]
                x_next, y_next = points[(i+1)%len(points)]
                
                # Elasticity component
                elastic_x = 2*alpha*(2*points[i][0] - x_prev - x_next)
                elastic_y = 2*alpha*(2*points[i][1] - y_prev - y_next)
                
                # Curvature component
                curve_x = 2*beta*(6*points[i][0] - 4*(x_prev + x_next) + (points[(i-2)%len(points)][0] + points[(i+2)%len(points)][0]))
                curve_y = 2*beta*(6*points[i][1] - 4*(y_prev + y_next) + (points[(i-2)%len(points)][1] + points[(i+2)%len(points)][1]))
                
                # Combined update (only moves toward significant edges)
                new_points[i][0] = points[i][0] - gamma*(gx + elastic_x + curve_x)
                new_points[i][1] = points[i][1] - gamma*(gy + elastic_y + curve_y)
            
            # Constrain points to image dimensions
            new_points[:,0] = np.clip(new_points[:,0], 0, self.width-1)
            new_points[:,1] = np.clip(new_points[:,1], 0, self.height-1)
            
            # Visualization
            vis = cv2.cvtColor(self.image, cv2.COLOR_GRAY2BGR)
            
            # Show thresholded edges for reference
            edges_vis = cv2.normalize(self.grad_mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            edges_vis = cv2.applyColorMap(edges_vis, cv2.COLORMAP_JET)
            vis = cv2.addWeighted(vis, 0.7, edges_vis, 0.3, 0)
            
            cv2.polylines(vis, [np.array(points, dtype=np.int32)], True, (0,255,255), 2)
            for (x,y) in points:
                cv2.circle(vis, (int(x),int(y)), 3, (0,255,0), -1)
            
            cv2.putText(vis, f"Iter: {iteration} Energy: {total:.1f}", (10,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            cv2.putText(vis, f"Grad Thresh: {self.gradient_threshold}", (10,60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            cv2.imshow('Active Contour', vis)
            if cv2.waitKey(30) == 27:
                break
            
            # Check convergence
            if iteration > 10 and np.linalg.norm(new_points - points) < 0.5:
                print(f"Converged at iteration {iteration}")
                break
                
            points = new_points.copy()
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return points, energy_history

# Load and process image
image = cv2.imread(r"C:\Users\nasir\OneDrive\Pictures\Screenshots\Screenshot 2025-06-18 092112.png", cv2.IMREAD_GRAYSCALE)
if image is None:
    print("Error loading image")
else:
    # Parameters - adjust gradient_threshold as needed (higher = only strongest edges)
    ac = ActiveContour(image, gradient_threshold=120)  # Try values between 20-50
    final_contour, energies = ac.optimize(
        n_points=80,
        alpha=0.5,  # Lower for more flexible spacing
        beta=0.7,   # Higher for smoother contours
        gamma=0.05, # Learning rate
        max_iter=3000
    )
    
    # Show final result
    result = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.polylines(result, [np.array(final_contour, dtype=np.int32)], True, (0,255,0), 2)
    cv2.imshow('Final Contour', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()